original (bad) borate code, calculation of revelle factor code
code to make maps

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

## Notebook overview

    Aim 1: compile references for buffer code (section 1)
    Aim 2: Trying to build code calculate & plot all 6 buffer factors calculated in Egleston 2010, as well as the classical Revelle factor. I can't reproduce the values they get in their paper and am unsure why - see sections 4-8. 




## 1. References

{Egleston 2010}
Since Revelle and Suess first introduced the concept of an
oceanic carbon buffer, several methods have been used for
calculating or estimating R [Broecker et al., 1979;
Sarmiento and Gruber, 2006]. Broecker et al. stated that
the Revelle factor is a function of the ratio of DIC to
alkalinity. By examining the explicit expression of R, we
can demonstrate that this is mathematically true under the
assumption that the minor species borate, proton, and
hydroxide can be neglected compared to bicarbonate. 

### Reference: buffer factors as derived by Egleston 2010:
<img src="./Egleston_Table1.png">


### DIC and TA proportional makeup reference, Gruber-Sarmiento Ch8:

<img src="./Sarmiento_percentages.png">

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
import sys
#sys.path.append('/data/tjarniko/MEOPAR/mocsy')
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/VIZ')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/')
import mocsy
import VIZ_plots as VP

import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# from oct2py import octave
import seawater as sw
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt
import glob

import matplotlib.pyplot as plt
from matplotlib import reload
plt.style.use('seaborn-bright')
import netCDF4 as nc
import numpy as np
import cmocean as cm
import CCCma_stations as STN
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

%matplotlib inline

ModuleNotFoundError: No module named 'CCCma_stations'

## 2) output visualisation: load data to work with and check it looks ok

    -from spinup year, 13 Nov 2014
    -use CCCma plotcode to check that visuals look ok 



In [ ]:
## Retreive one dataset 
date = 'Nov 13, 2014'
daypath = '/results/SalishSea/spinup.201812/13nov14/'
ptrc = 'SalishSea_1d_*_carp_T.nc'
grid = 'SalishSea_1d_*_grid_T.nc'
ptt = daypath  + ptrc
ptg = daypath + grid
pttG = glob.glob(ptt)
print(pttG)
ptgG = glob.glob(ptg)

carp_nc = nc.Dataset(pttG[0])
grid_nc = nc.Dataset(ptgG[0])


In [ ]:
# print(carp_nc.variables['dissolved_inorganic_carbon'])
print(grid_nc.variables['vosaline'])

In [ ]:
print(mocsy.mvars.vars.__doc__)

In [ ]:
def surface_maps(carp,grid,stns, ddmmmyy, rdir,humandate, dss_sig):
    
    tsal = grid.variables['vosaline'][0,0,:,:]
    ttemp = grid.variables['votemper'][0,0,:,:]
    tdic = carp.variables['dissolved_inorganic_carbon'][0,0,:,:]
    tta = carp.variables['total_alkalinity'][0,0,:,:]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-6
    tdra = np.ravel(tdic) * 1e-6
    tzero = np.zeros_like(tsra)

    response_tup = mocsy.mvars(temp=ttera, sal=tsra, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tzero, depth=tzero, lat=tzero, 
                        optcon='mol/kg', optt='Tinsitu', optp='db',
                        optb = 'u74', optk1k2='m10', optkf = 'pf', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pHr = pH.reshape(898,398)
    OmA = OmegaA.reshape(898,398)
    
    surf_dat = [tsal, tdic, tta, ttemp, pHr, OmA]
    
    vmins = [25,1800,1800,5,7.5,0]
    vmaxs = [32,2200,2200,15,8.5,2]
    msk = [0,0,0,0,1e20,1e20]
    cl = ['salinity psu', 'DIC umol/kg', 'TA umol/kg', 'temp deg C', 'pH', 'Omega A']
    t_cmap = [cm.cm.haline, cm.cm.matter, cm.cm.matter, cm.cm.thermal, cm.cm.speed, cm.cm.curl]

    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = \
    plt.subplots(figsize=(20, 27) , nrows=2, ncols=3)

    viz_tools.set_aspect(ax1)
    viz_tools.set_aspect(ax2)
    viz_tools.set_aspect(ax3)
    viz_tools.set_aspect(ax4)
    viz_tools.set_aspect(ax5)
    viz_tools.set_aspect(ax6)

    i = 0
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax1.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax1)
    cbar.set_label(cl[i], fontsize = 20 )

    i = 1
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax2.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax2)
    cbar.set_label(cl[i], fontsize = 20 )

    i = 2
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax3.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax3)
    cbar.set_label(cl[i], fontsize = 20 )

    i = 3
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax4.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax4)
    cbar.set_label(cl[i], fontsize = 20 )

    i = 4
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax5.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax5)
    cbar.set_label(cl[i], fontsize = 20 )

    i = 5
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,msk[i])
    tcmap = t_cmap[i]
    mesh = ax6.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax6)
    cbar.set_label(cl[i], fontsize = 20 )

    st = 'Salish Sea Carbonate Chemistry Map, ' + humandate
    plt.suptitle(st,fontsize=20)

    plt.show()

In [ ]:
surface_maps(carp_nc,grid_nc,STN, 'ddmmmyy' , 'rdir',date, 'dss_sig')

## 3) Point check of mocsy output - JdF waters y=400, x=50

Make sure that the values mocsy puts out aren't garbage/are plausible - yes. (We need mocsy output to calculate buffer factors.) At present there is no phosphate, silicate, and the patm = 0. At patm =1, these mocsy outputs are very similar - 3 uatm difference for pco2, speciation and pH values very similar indeed. 

        Mocsy output at patm = 0
        ~~~MOCSY OUTPUT~~
        pH: [ 7.84746113]
        pco2: [ 634.13513064]
        co2: [  2.97835710e-05]
        hco3: [ 0.0019442]
        co3: [  7.18200274e-05]
        OmegaA: [ 1.10299131]
        TA_carb from mocsy: [ 0.00208784]
        
        Mocsy output at patm = 1
        ~~~MOCSY OUTPUT~~
        pH: [ 7.84742219]
        pco2: [ 637.84197069]
        co2: [  2.97831492e-05]
        hco3: [ 0.0019442]
        co3: [  7.18186151e-05]
        OmegaA: [ 1.10277529]
        TA_carb from mocsy: [ 0.00208784]

In [ ]:
y = 400
x = 50
tsal = grid_nc.variables['vosaline'][0,0,y,x]
ttemp = grid_nc.variables['votemper'][0,0,y,x]
tdic = carp_nc.variables['dissolved_inorganic_carbon'][0,0,y,x]
tta = carp_nc.variables['total_alkalinity'][0,0,y,x]

tsra = np.ravel(tsal)
ttera = np.ravel(ttemp)
ttara = np.ravel(tta) * 1e-6
tdra = np.ravel(tdic) * 1e-6
tzero = np.zeros_like(tsra)
tone = np.zeros_like(tsra)
tone[:] = 1

sal = tsra
dic = tdra
alk = ttara

print('sal: ' + str(tsra))
print('temp: ' + str(ttera))
print('dic: ' + str(tdra))
print('ta: ' + str(ttara))

response_tup = mocsy.mvars(temp=ttera, sal=tsra, alk=ttara, dic=tdra, 
                   sil=tzero, phos=tzero, patm=tone, depth=tone, lat=tzero, 
                    optcon='mol/kg', optt='Tinsitu', optp='db',
                    optb = 'u74', optk1k2='m10', optkf = 'pf', optgas = 'Pinsitu')
pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

print('~~~MOCSY OUTPUT~~')
print('pH: ' + str(pH))
print('pco2: ' + str(pco2))
print('co2: ' + str(co2))
print('hco3: ' + str(hco3))
print('co3: ' + str(co3))
print('OmegaA: ' + str(OmegaA))


TA_carb = hco3 +2*co3
print('TA_carb from mocsy: '+str(TA_carb))




## 4) calculating missing parameters for the buffer factor calc: B(OH)4, h+, oh-

We have 3 parameters that we need for the BF calculations and that we don't currently have - borate ion, h+, oh-. This code calculates them and prints them out for a check - still intentionally using open-ocean values (edge of JdF)

I have some issues with this:

    1) Using 2 separate implementations of the Uppstrom equation, the borate concentration is really high - like 15%, as opposed to the 5% globally. Probably ok? 
    2) I end up getting a negative OH- concentration any way I slice it.  
    TA = [bicarbonate] + 2*[carbonate] + [B(OH)4-] + [OH-] - [H+]
    ergo
    [OH-] = TA - [bicarbonate] - 2*[carbonate] - [B(OH)4-] + [H+]
    This makes mathematical sense, but not physical sense. It's trivial math. I coded it right. mocsy is giving values I think are right. how does this make sense?
    
    I then use this code without the printouts directly in the buffer factor calculation code. 

In [ ]:
#hth do you get an expression for the hydroxide ion?

def calc_borate_hpl_ohminus(dic,alk,co2,hco3,co3,sal,pH):

    print('total ta: '+str(alk))
    print(' ')
    bicarb = hco3
    carb = co3
    #calculate borate, Uppstrom, 1974, looked up in mocsy
    scl=sal/1.80655
    borat = 0.000232  * scl/10.811
    print('borat conc (rel. from mocsy): ' + str(borat))
    print('proportion of TA: ' + str(borat/alk))
    borat2 = .0000119*sal
    print('borat conc (rel. from co2sys): ' + str(borat2))
    print('proportion of TA: ' + str(borat2/alk))
    print(' ')
    hplus = 10**(-1*pH)
    print('hplus conc: ' + str(hplus))
    print('proportion of TA: ' + str(hplus/alk))
    print(' ')
    

    
    ohminus = alk  - bicarb - 2*carb - borat  + hplus
    print('ohminus conc: ' + str(ohminus))
    print('proportion of TA: ' + str(ohminus/alk))
    print(' ')    
    print('bicarb conc: ' + str(bicarb))
    print('proportion of TA: ' + str(bicarb/alk))
    print(' ')
    print('carb conc (charge, double): ' + str(2*carb))
    print('proportion of TA: ' + str((2*carb)/alk))
    print(' ')
    
    dic = hco3+co3+co2
    print('bicarb proportion of DIC: '+str((bicarb/dic)))
    print('carb proportion of DIC: '+str((carb/dic)))
    print('co2 proportion of DIC: '+str((co2/dic)))
    

In [ ]:
calc_borate_hpl_ohminus(dic,alk,co2,hco3,co3,sal,pH)

## 5) Equations for the buffer factor code

K_hb is the "acidity constant for boric acid". Value obtained from the NIH reference site. 
Equations coded below - I did double-check them, I can't find any errors. [Is x equivalent to * ?] When I try to reproduce Figure 2 from Egleston, it doesn't work.

<img src="./Egleston_Table1.png">

In [ ]:

def calc_factors(dic,alk,co2,hco3,co3,sal,pH):

    bicarb = hco3
    carb = co3
    #calculate borate, Uppstrom, 1974, looked up in mocsy
    scl=sal/1.80655
    borat = 0.000232  * scl/10.811
    hplus = 10**(-1*pH)
    borat2 = .0000119*sal
    ohminus = alk  - bicarb - 2*carb - borat 

    # - calculates quantities needed for Egleston's factors, and the factors themselves
    
    #Khb is the acidity constant for boric acid - is this what they are talking about? 
    #https://pubchem.ncbi.nlm.nih.gov/compound/boric_acid#section=Decomposition
    Khb = 5.80e-10 

    
    S = bicarb + 4*(carb) + (hplus*borat)/(Khb+hplus)+hplus-ohminus
    P = 2*(carb)+bicarb
    AlkC = bicarb + 2*(carb)
    
    DIC = co2+bicarb+carb
    Alk = bicarb + 2*carb + borat - hplus + ohminus
   
    g_dic = DIC - AlkC**2/S
    b_dic = (DIC*S-AlkC**2)/AlkC
    w_dic = DIC - (AlkC*P)/bicarb
    
    g_alk = (AlkC**2-DIC*S)/AlkC
    b_alk = (AlkC**2/DIC)-S
    w_alk = AlkC - (DIC*bicarb)/P
    
    return g_dic, b_dic, w_dic, g_alk, b_alk, w_alk
    

## 6) Reproduce Figure 2 of Egleston

Egleston doesn't give T,S,P at which this was plotted, but they shouldn't matter. Needless to say I do not reproduce his figure with this code. 


<img src="./Egleston_fig2.png">

In [ ]:
#ft = figure test 

ft_temp = np.zeros([16])
ft_temp[:] = 10
ft_sal = np.zeros([16])
ft_sal[:] = 32
ft_alk = np.zeros([16])
ft_alk[:] = 0.00225
ft_dic = np.arange(0.0016,0.0032,0.0001)
ft_zero = np.zeros([16])
ft_patm = np.zeros([16])
ft_patm[:] = 1


response_tup = mocsy.mvars(temp=ft_temp, sal=ft_sal, alk=ft_alk, dic=ft_dic, 
                   sil=ft_zero, phos=ft_zero, patm=ft_patm, depth=ft_zero, lat=ft_zero, 
                    optcon='mol/kg', optt='Tinsitu', optp='db',
                    optb = 'u74', optk1k2='m10', optkf = 'pf', optgas = 'Pinsitu')
ft_pH,ft_pco2,ft_fco2,ft_co2,ft_hco3,ft_co3,ft_OmegaA,ft_OmegaC,ft_BetaD,ft_DENis,ft_p,ft_Tis = response_tup

ft_g_dic, ft_b_dic, ft_w_dic, ft_g_alk, ft_b_alk, ft_w_alk = calc_factors(ft_dic,ft_alk,ft_co2,ft_hco3,ft_co3,ft_sal,ft_pH)

In [ ]:
fig = plt.figure()
ax = plt.axes()

dic_umol = ft_dic*1000000 
ax.plot(dic_umol,ft_g_dic*1000);
ax.plot(dic_umol,ft_b_dic*1000);
ax.plot(dic_umol,-ft_w_dic*1000);
ax.plot(dic_umol,-ft_g_alk*1000);
ax.plot(dic_umol,-ft_b_alk*1000);
ax.plot(dic_umol,ft_w_alk*1000);
legtext = ['ft_g_dic', 'ft_b_dic', 'ft_w_dic', '-ft_g_alk', '-ft_b_alk', 'ft_w_alk']
ax.legend(legtext)

## 7) buffer factor mapping code

It's very pretty; if only the buffer factors were right. yellow and red are top/bottom 20% of the range I established based on nov 13 output. 

In [ ]:
def surface_buffer_maps(carp,grid, ddmmmyy, rdir,humandate, dss_sig):
    
    #retrieve relevant data for mocsy calculation, calculate mocsy
    tsal = grid.variables['vosaline'][0,0,:,:]
    ttemp = grid.variables['votemper'][0,0,:,:]
    tdic = carp.variables['dissolved_inorganic_carbon'][0,0,:,:]
    tta = carp.variables['total_alkalinity'][0,0,:,:]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-6
    tdra = np.ravel(tdic) * 1e-6
    tzero = np.zeros_like(tsra)
    

    response_tup = mocsy.mvars(temp=ttera, sal=tsra, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tzero, depth=tzero, lat=tzero, 
                        optcon='mol/kg', optt='Tinsitu', optp='db',
                        optb = 'u74', optk1k2='m10', optkf = 'pf', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    #calculate borate and ohminus concentration
    
    bicarb = hco3
    carb = co3
    #calculate borate, Uppstrom, 1974, looked up in mocsy
    scl=sal/1.80655
    borat = 0.000232  * scl/10.811
    hplus = 10**(-1*pH)
    borat2 = .0000119*sal
    ohminus = alk  - bicarb - 2*carb - borat 

    # - calculates quantities needed for Egleston's factors, and the factors themselves
    
    #Khb is the acidity constant for boric acid - is this an appropriate ref?
    # https://www2.chemistry.msu.edu/courses/cem262/aciddissconst.html
    Khb = 5.81e-10 

    
    S = bicarb + 4*(carb) + (hplus*borat)/(Khb+hplus)+hplus-ohminus
    P = 2*(carb)+bicarb
    AlkC = bicarb + 2*(carb)
    
    DIC = co2+bicarb+carb
    Alk = bicarb + 2*carb + borat - hplus + ohminus
   
    g_dic = DIC - AlkC**2/S
    b_dic = (DIC*S-AlkC**2)/AlkC
    w_dic = DIC - (AlkC*P)/bicarb
    
    g_alk = (AlkC**2-DIC*S)/AlkC
    b_alk = (AlkC**2/DIC)-S
    w_alk = AlkC - (DIC*bicarb)/P
    
    #### 
    g_dicR = g_dic.reshape(898,398) *1000
    b_dicR = b_dic.reshape(898,398) *1000
    w_dicR = w_dic.reshape(898,398) *-1000
    g_alkR = g_alk.reshape(898,398) *-1000
    b_alkR = b_alk.reshape(898,398) *-1000
    w_alkR = w_alk.reshape(898,398) *1000

    #this bit figures out ranges
    #print(b_dicR[500,0:20])
#     print(np.max(g_dicR))
#     print(np.max(b_dicR))
#     print(np.min(w_dicR))
#     print(np.min(g_alkR))
#     print(np.min(b_alkR))
#     print(np.max(w_alkR))

#     print('Max min gdic')
#     print(g_dicR[420,310])
#     print(g_dicR[420,20])
#     print('Max min bdic')
#     print(b_dicR[420,310])
#     print(b_dicR[420,20])
#     print('Max min wdic')
#     print(w_dicR[420,310])
#     print(w_dicR[420,20])
#     print('Max min galk')
#     print(g_alkR[420,310])
#     print(g_alkR[420,20])
#     print('Max min balk')
#     print(b_alkR[420,310])
#     print(b_alkR[420,20])
#     print('Max min walk')
#     print(w_alkR[420,310])
#     print(w_alkR[420,20])
    surf_dat = [g_dicR, b_dicR, w_dicR, g_alkR, b_alkR, w_alkR]
    
    vmins = [-0.7,-0.4,-0.1,-0.4,-0.4,-0.1]
    vmaxs = [0.7,1,0.5,1,1,0.4]
    msk = [1.875e+23,5e+23,6e+23,5e+23,5e+23,2e+23] 

    cl = ['$\gamma_{DIC}$', '$\\beta_{DIC}$', '-$\omega_{DIC}$',\
          '$\gamma_{TA}$', '$\\beta_{TA}$', '-$\omega_{TA}$']
     
    #t_cmap = [cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep]
    t_cmap = [cm.cm.oxy, cm.cm.oxy, cm.cm.oxy, cm.cm.oxy, cm.cm.oxy, cm.cm.oxy ]
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = \
    plt.subplots(figsize=(20, 27) , nrows=2, ncols=3)

    viz_tools.set_aspect(ax1)
    viz_tools.set_aspect(ax2)
    viz_tools.set_aspect(ax3)
    viz_tools.set_aspect(ax4)
    viz_tools.set_aspect(ax5)
    viz_tools.set_aspect(ax6)

    i = 0
    #'g_dicR',
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,1.875e+23)
    tcmap = t_cmap[i]
    mesh = ax1.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax1)
    cbar.set_label(cl[i], fontsize = 20 )
    ax1.set_title('$CO_{2}$ with DIC',fontsize = 22)
    
    i = 1
    #'b_dicR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,5e+23)
    tcmap = t_cmap[i]
    mesh = ax2.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax2)
    cbar.set_label(cl[i], fontsize = 20 )
    ax2.set_title('pH with DIC',fontsize = 22)

    i = 2
    #'-w_dicR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,6e+23)
    tcmap = t_cmap[i]
    mesh = ax3.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax3)
    cbar.set_label(cl[i], fontsize = 20 )
    ax3.set_title('$\Omega$ with DIC',fontsize = 22)

    i = 3
    #'-g_alkR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,5e+23)
    tcmap = t_cmap[i]
    mesh = ax4.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax4)
    cbar.set_label(cl[i], fontsize = 20 )
    ax4.set_title('$CO_{2}$ with TA',fontsize = 22)
    
    
    i = 4
    #'-b_alkR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,5e+23)
    tcmap = t_cmap[i]
    mesh = ax5.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax5)
    cbar.set_label(cl[i], fontsize = 20 )
    ax5.set_title('pH with TA',fontsize = 22)

    i = 5
    #'w_alkR'
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,2e+23)
    tcmap = t_cmap[i]
    mesh = ax6.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax6)
    cbar.set_label(cl[i], fontsize = 20 )
    ax6.set_title('$\Omega$ with TA',fontsize = 22)

    #tcmap.set_bad('white')
    st = 'Carbonate Chemistry Buffer Factors, ' + humandate
    plt.suptitle(st,fontsize=20)

    plt.show()

In [ ]:
surface_buffer_maps(carp_nc,grid_nc, 'ddmmmyy', 'rdir', 'humandate', 'dss_sig')

## 8) code to visualise Revelle factor R

As expected, because $\gamma$DIC is off from values that make sense, so is R - R here is DIC/$\gamma$DIC. As a check, contains 2 visualisations of DIC - DIC from model output and DIC from summation of mocsy outputs. They're the same, which makes sense. 

In [ ]:
def surface_Revelle_map(carp,grid, ddmmmyy, rdir,humandate, dss_sig):
    
    #retrieve relevant data for mocsy calculation, calculate mocsy
    tsal = grid.variables['vosaline'][0,0,:,:]
    ttemp = grid.variables['votemper'][0,0,:,:]
    tdic = carp.variables['dissolved_inorganic_carbon'][0,0,:,:]
    tta = carp.variables['total_alkalinity'][0,0,:,:]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-6
    tdra = np.ravel(tdic) * 1e-6
    tzero = np.zeros_like(tsra)
    

    response_tup = mocsy.mvars(temp=ttera, sal=tsra, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tzero, depth=tzero, lat=tzero, 
                        optcon='mol/kg', optt='Tinsitu', optp='db',
                        optb = 'u74', optk1k2='m10', optkf = 'pf', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    #calculate borate and ohminus concentration
    
    bicarb = hco3
    carb = co3
    #calculate borate, Uppstrom, 1974, looked up in mocsy
    scl=sal/1.80655
    borat = 0.000232  * scl/10.811
    hplus = 10**(-1*pH)
    borat2 = .0000119*sal
    ohminus = alk  - bicarb - 2*carb - borat 

    # - calculates quantities needed for Egleston's factors, and the factors themselves
    
    #Khb is the acidity constant for boric acid - is this an appropriate ref?
    # https://www2.chemistry.msu.edu/courses/cem262/aciddissconst.html
    Khb = 5.81e-10 

    
    S = bicarb + 4*(carb) + (hplus*borat)/(Khb+hplus)+hplus-ohminus
    P = 2*(carb)+bicarb
    AlkC = bicarb + 2*(carb)
    
    DIC = co2+bicarb+carb
    Alk = bicarb + 2*carb + borat - hplus + ohminus
   
    g_dic = DIC - AlkC**2/S
    b_dic = (DIC*S-AlkC**2)/AlkC
    w_dic = DIC - (AlkC*P)/bicarb
    
    g_alk = (AlkC**2-DIC*S)/AlkC
    b_alk = (AlkC**2/DIC)-S
    w_alk = AlkC - (DIC*bicarb)/P
    
    DIC_mocsy = DIC.reshape(898,398)*1000000
    
    
    #### 
    g_dicR = g_dic.reshape(898,398) 

    R = ((tdic/1e6)/g_dicR)

    #lots of test printouts -sanity checks
#     print('r fill')
#     print(R[0,0])
#     
#     print('mocsyfill')
#     print(DIC_mocsy[0,0])
#     print(tdic[250,250]/1e6)
#     print('test')
#     a = tdic[250,250]/1e6
#     print(a)
#     b = g_dicR[250,250]
#     print(b)
#     print('rev')
#     print(a/b)

#     print(np.min(R))
#     print(np.max(R))
    
    surf_dat = [tdic, DIC_mocsy, R]
    vmins = [0,0,0]
    vmaxs = [2200,2200,12]
    msk = [1.875e+23,3e+26,6e+23] 

    cl = ['umol/kg','umol/kg','R']
     
    #t_cmap = [cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep, cm.cm.deep]
    t_cmap = [cm.cm.thermal, cm.cm.thermal, cm.cm.thermal ]
    fig, ((ax1, ax2, ax3)) = \
    plt.subplots(figsize=(20, 13) , nrows=1, ncols=3)

    viz_tools.set_aspect(ax1)
    viz_tools.set_aspect(ax2)
    viz_tools.set_aspect(ax3)


    i = 0
    #'g_dicR',
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,1.875e+23)
    tcmap = t_cmap[i]
    mesh = ax1.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax1)
    cbar.set_label(cl[i], fontsize = 20 )
    ax1.set_title('DIC - from SKOG',fontsize = 22)
    
    i = 1
    #'b_dicR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,3e26)
    
    tcmap = t_cmap[i]
    mesh = ax2.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax2)
    cbar.set_label(cl[i], fontsize = 20 )
    ax2.set_title('DIC - from mocsy (check)',fontsize = 22)

    i = 2
    #'-w_dicR', 
    tplt0 = surf_dat[i]
    tplt = np.ma.masked_values(tplt0,6e+23)
    tcmap = t_cmap[i]
    mesh = ax3.pcolormesh(tplt, cmap=tcmap, vmin=vmins[i], vmax=vmaxs[i])
    cbar = fig.colorbar(mesh, ax=ax3)
    cbar.set_label(cl[i], fontsize = 20 )
    ax3.set_title('Revelle factor R',fontsize = 22)



    #tcmap.set_bad('white')
    st = 'DIC and Revelle factor, ' + humandate
    plt.suptitle(st,fontsize=20)

    plt.show()

In [ ]:
surface_Revelle_map(carp_nc,grid_nc, 'ddmmmyy', 'rdir', 'Nov 13 2014', 'dss_sig')

## for reference - code calculating borate

##### CO2SYS:
    ## CO2SYS.m, line 692
    
    	FF=F&(WhoseKSO4==1|WhoseKSO4==2); % If user opted for Uppstrom's values:
	if any(FF)
	    % Uppstrom, L., Deep-Sea Research 21:161-162, 1974:
	    % this is .000416.*Sali./35. = .0000119.*Sali
		% TB(FF) = (0.000232./10.811).*(Sal(FF)./1.80655); % in mol/kg-SW
	    TB(FF) =  0.0004157.*Sal(FF)./35; % in mol/kg-SW
	end
	FF=F&(WhoseKSO4==3|WhoseKSO4==4); % If user opted for the new Lee values:
	if any(FF)
		% Lee, Kim, Byrne, Millero, Feely, Yong-Ming Liu. 2010.	
	 	% Geochimica Et Cosmochimica Acta 74 (6): 1801â1811.
		TB(FF) =  0.0004326.*Sal(FF)./35; % in mol/kg-SW
        
##### mocsy:
    ## mocsy_mainmod.F90, line 888
    !       Salinity and simply related values
        s = DBLE(ssal)
        s2=s*s
        sqrts=SQRT(s)
        s15=s**1.5d0
        scl=s/1.80655d0

    !       Ionic strength:
            is = 19.924d0*s/(1000.0d0 - 1.005d0*s)
            is2 = is*is
            sqrtis = SQRT(is)

    !       Total concentrations for sulfate, fluoride, and boron

    !       Sulfate: Morris & Riley (1966)
            St(i) = 0.14d0 * scl/96.062d0

    !       Fluoride:  Riley (1965)
            Ft(i) = 0.000067d0 * scl/18.9984d0

    !       Boron:
            IF (trim(opB) == 'l10') THEN
    !          New formulation from Lee et al (2010)
               Bt(i) = 0.0002414d0 * scl/10.811d0
            ELSEIF (trim(opB) == 'u74') THEN
    !          Classic formulation from Uppström (1974)
               Bt(i) = 0.000232d0  * scl/10.811d0